In [1]:
import numpy as np
import pandas as pd
import nltk
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
# nltk.download('tagsets')

In [66]:
#assuming text is preprocessed then applying these
def num_of_words(text):
    return len(text.split())

def average_word_length(text):
    return len(text)/num_of_words(text)

def pos_tagging(text):
    #get the frequency of each tag after NER
    words=text.split()
    tagged=nltk.pos_tag(words)
    result=[]
    freq=defaultdict(int)
    for word, noun in tagged:
        freq[noun]+=1
    return freq

#maybe use the average score over previous X tests. could be a good feature for clustering
# def get_average_score():
#     pass

vectorizer = TfidfVectorizer(max_df=0.85, min_df=1, max_features=5000, stop_words='english')


# documentA = 'the man went out for a walk and then sat'
# documentB = 'the children sat around the fire'
# vectors = vectorizer.fit_transform([documentA, documentB])
# vectorizer.get_feature_names()
def generate_vectors(documents):
#     print(vectorizer.max_df)
    vectorizer.fit(documents)
    return vectorizer.transform(documents)

In [67]:
pos_tagging("Hello world, from India! How are you doing?")

defaultdict(int, {'NNP': 3, 'NN': 1, 'IN': 1, 'VBP': 2, 'PRP': 1})

In [68]:
average_word_length("Hello world, from India! How are you doing?")

5.375

In [69]:
#https://github.com/plasticityai/magnitude#using-the-library
from pymagnitude import *
vectors = Magnitude("glove-lemmatized.6B.100d.magnitude")

In [70]:
len(vectors)

336951

In [71]:
vectors.query("technology")

array([-0.0194448,  0.1029267,  0.0693665,  0.0018058,  0.0794503,
       -0.2790355, -0.0184297, -0.0963583,  0.07153  ,  0.0857471,
        0.0297684, -0.1342948, -0.0147681, -0.0501425,  0.0175402,
       -0.0753504,  0.1229656,  0.0498725, -0.0186124, -0.0211477,
       -0.11231  , -0.0957562,  0.1035812,  0.0289361,  0.0347452,
        0.0472991,  0.0886731,  0.0249267, -0.1134489,  0.0550827,
       -0.1398767,  0.1949245, -0.1753384,  0.0122737,  0.0216179,
       -0.0596497, -0.0874214, -0.0324387, -0.0392009, -0.0590889,
       -0.0621913, -0.1211229, -0.126797 , -0.0517564, -0.126689 ,
       -0.0972399,  0.081887 ,  0.1497683,  0.0316063, -0.0867621,
       -0.0007265,  0.0148413, -0.013114 ,  0.1139668,  0.0566093,
       -0.2743653, -0.051048 ,  0.0340383,  0.3492153, -0.049998 ,
       -0.0215654,  0.0475135, -0.0243294,  0.0678574, -0.0370739,
        0.0505841,  0.083798 ,  0.0059875,  0.1437273,  0.1300566,
        0.0436217,  0.0419268, -0.0219578, -0.0723243, -0.0898

In [78]:
answers=[
    'the combination of a clash symbol be accompany by a bass drum play an dent part played loudly',
    'it strategy location and excellent infrastructure with the largest airport in scandinavian local 14 minutes by train from the city centre',
    'a threshold flute made from a mammoth tusk and two flutes made from swans bones are among the oldest known music instruments']
try:
    tfidf_vec=generate_vectors(answers)
except ValueError:
    vectorizer=TfidfVectorizer(max_df=1.00, min_df=1, max_features=5000, stop_words='english')
    tfidf_vec=generate_vectors(answers)

1.0


In [154]:
answers=[
    'delhi is the capital of india',
    'the capital of india is delhi',
    'this is something unrelated']
try:
    tfidf_vec=generate_vectors(answers)
except ValueError:
    vectorizer=TfidfVectorizer(max_df=1.00, min_df=1, max_features=5000)
    tfidf_vec=generate_vectors(answers)

1.0


In [155]:
vectorizer.get_feature_names()

['capital',
 'delhi',
 'india',
 'is',
 'of',
 'something',
 'the',
 'this',
 'unrelated']

In [156]:
def tfidf_weighted_avg_vec(para, idx):
    s=0
    for i, word in enumerate(para.split(' ')):
        weight=tfidf_vec[idx].toarray().flatten()[i]
        v=weight*vectors.query(word)
        s+=v
    s/=len(para.split(' '))
    return s

In [157]:
answers[0]

'delhi is the capital of india'

In [162]:
v1=tfidf_weighted_avg_vec(answers[0], 0)
v2=tfidf_weighted_avg_vec(answers[1], 1)
print(cosine_similarity([v1, v2]))

[[0.99999994 0.98323834]
 [0.98323834 1.        ]]
